# UPPP Week 7 - Accessibility

<a target="_blank" href="https://colab.research.google.com/github/knaaptime/uppp135-winter26-assn/blob/main/week7/accessibility.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
import pandas as pd
import geopandas as gpd
import pandarm as pdna
import geosnap as gsp

In [ ]:
# !pip install h3

In [ ]:
import osmnx as ox
from tobler.util import h3fy

In [ ]:
irvine_bound = ox.geocode_to_gdf('irvine, ca')

In [ ]:
irvine_bound.explore()

In [ ]:
irvine_hex = h3fy(irvine_bound, resolution=9, buffer=True)

In [ ]:
irvine_hex.explore()

In [ ]:
from geosnap.visualize import geopandas

## Data

In [ ]:
# this is a lot of data to download

net = pdna.Network.from_gdf(irvine_bound, network_type='all') 

In [ ]:
net.save_hdf5("irvine_net.h5")

In [ ]:
bound = irvine_bound.union_all()

In [ ]:
restaurants = ox.features_from_polygon(bound, tags={'amenity':'restaurant'})
restaurants = restaurants.set_geometry(restaurants.centroid)

In [ ]:
restaurants.explore()

In [ ]:
parks = ox.features_from_polygon(bound, tags={'leisure':'park'})
parks = parks.set_geometry(parks.centroid)

In [ ]:
parks.explore()

In [ ]:
clinics = parks = ox.features_from_polygon(bound, tags={'amenity':'clinic'})
clinics = clinics.set_geometry(clinics.centroid)

In [ ]:
clinics.explore(style_kwds={'radius':10})

In [ ]:
fire_stations = ox.features_from_polygon(bound, tags={'amenity':'fire_station'})
fire_stations = fire_stations.set_geometry(fire_stations.centroid)

In [ ]:
fire_stations.explore(style_kwds={'radius':10})

## Attaching to Network

In [ ]:
restaurants['node_ids'] = net.get_node_ids(restaurants.geometry.x, restaurants.geometry.y)
restaurants['restaurants'] = 1

restaurants

In [ ]:
clinics['node_ids'] = net.get_node_ids(clinics.geometry.x, clinics.geometry.y)
clinics['clinics'] = 1

In [ ]:
parks["node_ids"] = net.get_node_ids(parks.geometry.x, parks.geometry.y)
parks["parks"] = 1

In [ ]:
fire_stations['node_ids'] = net.get_node_ids(fire_stations.geometry.x, fire_stations.geometry.y)
fire_stations['fire_stations'] = 1

In [ ]:
net.set(restaurants['node_ids'], variable=restaurants['restaurants'], name="restaurants")

net.set(clinics.node_ids, variable=clinics['clinics'], name="clinics")

net.set(parks.node_ids, variable=parks['parks'], name="parks")

net.set(fire_stations.node_ids, variable=fire_stations['fire_stations'], name="fire_stations")

In [ ]:
acc_rest = net.aggregate(2000, decay='exp', name='restaurants')

In [ ]:
acc_rest

In [ ]:
acc_rest_500 = net.aggregate(500, decay='linear', name='restaurants')

In [ ]:
acc_park = net.aggregate(2000, decay='linear', name='parks')

In [ ]:
acc_fire_stations = net.aggregate(2000, decay='linear', name='fire_stations')

In [ ]:
acc_clinic = net.aggregate(2000, decay='linear', name='clinics')

## Attaching to Hexagons

In [ ]:
irvine_hex['node_ids'] = net.get_node_ids(irvine_hex.centroid.x, irvine_hex.centroid.y)

In [ ]:
irvine_hex.merge(acc_rest.rename('restaurants_2k'), left_on='node_ids', right_index=True)

In [ ]:
irvine_hex = irvine_hex.merge(acc_rest.rename('restaurants_2k'), left_on='node_ids', right_index=True)

In [ ]:
irvine_hex.explore('restaurants_2k', scheme='fisher_jenks', k=8, cmap='magma', tiles='cartodb positron', style_kwds={'weight':0.5})

In [ ]:
irvine_hex.merge(
    acc_rest_500.rename("rest500"), left_on="node_ids", right_index=True
).explore("rest500", scheme='fisher_jenks', k=8, cmap='magma', tiles='cartodb positron', style_kwds={'weight':0.5})

In [ ]:
irvine_hex = irvine_hex.merge(acc_fire_stations.rename('fire_stations'), left_on='node_ids', right_index=True)

In [ ]:
m =irvine_hex.explore('fire_stations', scheme='fisher_jenks', k=8, cmap='magma', tiles='cartodb positron', style_kwds={'weight':0.5})
fire_stations.explore(m=m)
m

In [ ]:
irvine_hex = irvine_hex.merge(acc_clinic.rename('clinics'), left_on='node_ids', right_index=True)

In [ ]:
m =irvine_hex.explore('clinics', scheme='fisher_jenks', k=8, cmap='magma', tiles='cartodb positron', style_kwds={'weight':0.5})
clinics.explore(m=m)
m

In [ ]:
## lets do parks

## More visualization

In [ ]:
#!pip install lonboard

In [ ]:
from mapclassify.util import get_color_array
from lonboard import viz

In [ ]:
colors = get_color_array(irvine_hex['restaurants_2k'], scheme='fisher_jenks', cmap='magma')

In [ ]:
colors

In [ ]:
height = irvine_hex['restaurants_2k'] ** 2

In [ ]:
height

In [ ]:
viz(irvine_hex, polygon_kwargs={'get_fill_color':colors, 'get_elevation':height, 'extruded':True})